In [1]:
import gensim
import numpy as np
import pandas as pd
from IPython import embed

from ggplot import *
import json
import logging
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pyLDAvis.gensim
import seaborn as sns
import warnings
import pickle
import model_visualization
from gensim.models import CoherenceModel, HdpModel

%matplotlib inline
pyLDAvis.enable_notebook()

#pd.options.display.max_rows = 10

/Users/ismglv/anaconda3/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/Users/ismglv/anaconda3/lib/python3.6/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/Users/ismglv/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
with open('text_corpus_id2word_fulldata.pkl', 'rb') as handle:
   texts, id2word, corpus, full_data = pickle.load(handle)
#  texts, id2word, corpus, full_data = generate_corpus.generate_corpus()
# embed()

# View
print(corpus[:1])

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]


ldamallet =  gensim.models.wrappers.LdaMallet.load('lda.model')
# mallet_path = '/Users/ismglv/dev/lda2vec/mallet/bin/mallet' # update this path
# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=60, id2word=id2word)
# # coherence_model = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
# coherence = coherence_model.get_coherence()
# print('\nCoherence Score: ', coherence)

dictionary = texts

df_topic_sents_keywords = model_visualization.format_topics_sentences(ldamallet, corpus, dictionary)

df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['doc_id', 'dom_topic', 'topic_weight', 'topic_words', 'Text']



# to get doc topics df_dominant_topic['Keywords'][doc_num]
# to get doc topics dominant quality df_dominant_topic['Dominant_Topic'][doc_num]

#Find the most representative document for each topic

# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['dom_topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['dom_topic', 'topic_words']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['dom_topic', 'topic_words', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

#pyLDAvis.gensim.prepare(model, corpus, id2word)

data = pd.merge(full_data, df_dominant_topic, left_index=True, right_index=True)

arrays_of_data = {}
data_values = data.values
for d in data_values:
    if d[7] in arrays_of_data:
        arrays_of_data[d[7]].append(d)
    else:
        arrays_of_data[d[7]] = []
        arrays_of_data[d[7]].append(d)

counter = 0
part = []
parted_data = []
for i in arrays_of_data:
    if counter == 9:
        part.append(arrays_of_data[i])
        parted_data.append(part)
        counter = 0
        part = []
    else:
        part.append(arrays_of_data[i])
        counter += 1

def merged_list(lists):
    merged_list = []

    for l in lists:
        merged_list += l
    return merged_list
parts = []

for i in parted_data:
    parts.append(merged_list(i))
    
    
def group_topics_by_date(data):
    topics = {}
    for topic in data:
        if topic[3] in topics:
            topics[topic[3]].append(topic)
        else:
            topics[topic[3]] = []
            topics[topic[3]].append(topic)
    return topics

def group_topics_by_topic(data):
    topics = {}
    for topic in data:
        if topic[7] in topics:
            topics[topic[7]].append(topic)
        else:
            topics[topic[7]] = []
            topics[topic[7]].append(topic)
    return topics

def get_avg_of_topic(topics):
    sum = 0
    for topic in topics:
        sum += topic[6]
    return round(sum / len(topics), 3)

def get_key(dict):
    return list(dict.keys())[0]

def get_graph_data(data):
    result = {"2017-02": [], "2017-03": [], "2017-04": [], "2017-05": [], "2017-06": [], "2017-07": [], "2017-08": []}
    topics_by_date = group_topics_by_date(data)
    for topic in topics_by_date:
        topics = topics_by_date[topic]
        for t in topics:
            sorted_by_topic = group_topics_by_topic(topics)
            print(topic)
            print(result[topic])
            print(t[7])
            print(sorted_by_topic[t[7]])
            result[topic].append({t[7]: get_avg_of_topic(sorted_by_topic[t[7]])})       
    return result   
a = get_graph_data(parts[0])

model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [11]:
cols = ['filename', 'words', 'date', 'date_short', 'doc_id', 'dom_topic',
       'topic_weight', 'topic_words', 'Text']
test = pd.DataFrame(parts[0], columns=cols)

In [13]:
test = test.groupby(["date_short", "topic_words"]).mean()
test.drop(columns=["doc_id", "dom_topic"])

In [ ]:
dates = ["2017-02", "2017-03", "2017-04", "2017-05", "2017-06", "2017-07", "2017-08"]


In [182]:
def get_key(dict):
    return list(dict.keys())[0]

In [183]:
get_key(a["2017-02"][0])

'water, fish, year, britain, sea, climate, climate_change, catch, british, fishing'